In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
import seaborn as sns
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import classification_report, accuracy_score, f1_score
import itertools

In [2]:
#file_path = r"C:\Users\jens.nilsen\OneDrive - Bouvet Norge AS\Documents\GitHub\trfkaipoc\data_2022-2025.csv"
file_path = r"C:\Users\jens.nilsen\OneDrive - Bouvet Norge AS\Documents\GitHub\trfkaipoc\data_2022-2025_norge.csv"
df = pd.read_csv(file_path, sep=";")

In [3]:
df = df[df["EGS.VEDTAK.10670"].notna()]

In [4]:
df['Avslag_ind'] = df['EGS.VEDTAK.10670'].apply(lambda x: 1 if x == "Avslag" else 0)

In [5]:
andel_avslag=df['Avslag_ind'].sum()/df['Avslag_ind'].count()

In [6]:
andel_avslag

0.030107526881720432

In [7]:
antall_avslag=df['Avslag_ind'].sum()

In [8]:
antall_avslag

28

In [9]:
df

,OBJ.VEGOBJEKT-ID,EGS.BRUKSOMRÅDE.1256,EGS.PRIMÆRAKTIVITET.1824,EGS.SIKTFORHOLD.12051,EGS.SAKSNUMMER.1822,EGS.SØKNAD MOTTATT DATO.12048,"EGS.ARKIVREFERANSE, URL.12050",EGS.VEDTAK.10670,EGS.VEDTAKSDATO.11444,EGS.TILLATELSE GJELDER TIL DATO.12049,...,"ÅDT, total","ÅDT, andel lange kjøretøy",Fartsgrense,"Avkjørsel, holdningsklasse",Funksjonsklasse,Avkjørsler,Trafikkulykker,"Kurvatur, horisontalelement","Kurvatur, stigning",Avslag_ind
0,1025223484,Bolig,NaN,4x100,2024/60677,14.05.2024,NaN,Varig tillatelse,19.02.2025,NaN,...,100.0,10.0,80.0,Lite streng,E - Lokale adkomstveger,0,0,-558.0,-2.7,0
1,1025236953,Bolig,NaN,6x45,2021/20963,05.01.2025,NaN,Varig tillatelse,08.01.2025,NaN,...,850.0,14.0,50.0,Lite streng,E - Lokale adkomstveger,0,4,-24.0,8.4,0
2,1025238261,Bolig,NaN,4x60,2023/71315,23.12.2023,NaN,Varig tillatelse,28.06.2024,NaN,...,800.0,10.0,60.0,Mindre streng,D - Lokale samleveger,0,0,99999.0,3.7,0
3,1025214056,Industri,NaN,6x45,2025/9557,14.01.2025,NaN,Midlertidig tillatelse,03.02.2025,31.12.2025,...,6400.0,12.0,50.0,Meget streng,C - Lokale hovedveger,0,23,99999.0,-0.7,0
4,1025236984,Bolig,NaN,4x45,2024/56258,23.04.2024,NaN,Varig tillatelse,13.05.2024,NaN,...,600.0,14.0,50.0,Lite streng,D - Lokale samleveger,1,2,99999.0,-1.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,1016361352,Bolig,NaN,NaN,202435243-2,08.07.2024,https://trondelag.public360online.com/locator/...,Avslag,06.09.2024,06.09.2027,...,1300.0,6.0,30.0,Lite streng,E - Lokale adkomstveger,21,0,-1520.0,-0.5,1
926,1022497081,Gardsbruk,NaN,NaN,202516520-2,21.03.2025,https://trondelag.public360online.com/locator/...,Varig tillatelse,02.04.2025,02.04.2028,...,500.0,10.0,60.0,Mindre streng,D - Lokale samleveger,12,2,-1119.0,0.2,0
927,1018366642,Bolig,NaN,NaN,202340342-4,03.01.2024,https://trondelag.public360online.com/locator/...,Varig tillatelse,08.01.2024,08.01.2027,...,400.0,13.0,60.0,Streng,C - Lokale hovedveger,12,0,-192.0,5.5,0
928,1018305147,Skog/skogbruk,NaN,NaN,202337403-4,10.10.2023,https://trondelag.public360online.com/locator/...,Varig tillatelse,25.10.2023,25.10.2026,...,1600.0,15.0,80.0,Streng,B - Regionale hovedveger,8,6,-524.0,-3.0,0


In [10]:
df["Kurvatur, horisontal"]=df["Kurvatur, horisontalelement"]

In [11]:
features = [
    'Avslag_ind',
    "ÅDT, total",
    "ÅDT, andel lange kjøretøy",
    "Fartsgrense",
    "Avkjørsel, holdningsklasse",
    "Funksjonsklasse",
    "Avkjørsler",
    "Trafikkulykker",
    "EGS.BRUKSOMRÅDE.1256", 
    "Kurvatur, horisontal", 
    "EGS.SIKTFORHOLD.12051",
    "Kurvatur, stigning"
]
df=df[features]
# Encode categorical features

In [12]:
# Funksjon for å trekke ut siktlengde
def extract_sikt(val):
    if pd.isna(val):
        return np.nan
    val = val.strip().lower()
    if 'god' in val:
        return None  # vil kategorisere som lang sikt
    # Finn alle tall i teksten
    match = re.findall(r'\d+', val)
    if match:
        sikt_m = max(int(x) for x in match)  # tar lengste som representativ
        return sikt_m
    return None

df['sikt_m'] = df['EGS.SIKTFORHOLD.12051'].apply(extract_sikt)

# Lag kategoriske variabler direkte
def kategoriser_sikt(val, original):
    if pd.isna(original):
        return 'ukjent sikt'
    if val is None or ('god' in str(original).lower()):
        return 'lang sikt'
    elif val < 50:
        return 'kort sikt'
    elif val <= 99:
        return 'middels sikt'
    else:
        return 'lang sikt'

#df['sikt_kategori'] = df.apply(lambda row: kategoriser_sikt(row['sikt_m'], row['EGS.SIKTFORHOLD.12051']), axis=1)
df = df.drop(['EGS.SIKTFORHOLD.12051','sikt_m'], axis=1)

C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_29148\3441133011.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sikt_m'] = df['EGS.SIKTFORHOLD.12051'].apply(extract_sikt)


In [13]:
df_encoded=pd.get_dummies(df)

In [14]:
df_encoded

,Avslag_ind,"ÅDT, total","ÅDT, andel lange kjøretøy",Fartsgrense,Avkjørsler,Trafikkulykker,"Kurvatur, horisontal","Kurvatur, stigning","Avkjørsel, holdningsklasse_Lite streng","Avkjørsel, holdningsklasse_Meget streng",...,EGS.BRUKSOMRÅDE.1256_Dyrkamark/jordbruk,EGS.BRUKSOMRÅDE.1256_Forretning,EGS.BRUKSOMRÅDE.1256_Fritidsaktivitet,EGS.BRUKSOMRÅDE.1256_Gardsbruk,EGS.BRUKSOMRÅDE.1256_Hytte,EGS.BRUKSOMRÅDE.1256_Industri,EGS.BRUKSOMRÅDE.1256_Naust,EGS.BRUKSOMRÅDE.1256_Service,EGS.BRUKSOMRÅDE.1256_Skog/skogbruk,EGS.BRUKSOMRÅDE.1256_Skole/Barnehage
0,0,100.0,10.0,80.0,0,0,-558.0,-2.7,True,False,...,False,False,False,False,False,False,False,False,False,False
1,0,850.0,14.0,50.0,0,4,-24.0,8.4,True,False,...,False,False,False,False,False,False,False,False,False,False
2,0,800.0,10.0,60.0,0,0,99999.0,3.7,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0,6400.0,12.0,50.0,0,23,99999.0,-0.7,False,True,...,False,False,False,False,False,True,False,False,False,False
4,0,600.0,14.0,50.0,1,2,99999.0,-1.8,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,1,1300.0,6.0,30.0,21,0,-1520.0,-0.5,True,False,...,False,False,False,False,False,False,False,False,False,False
926,0,500.0,10.0,60.0,12,2,-1119.0,0.2,False,False,...,False,False,False,True,False,False,False,False,False,False
927,0,400.0,13.0,60.0,12,0,-192.0,5.5,False,False,...,False,False,False,False,False,False,False,False,False,False
928,0,1600.0,15.0,80.0,8,6,-524.0,-3.0,False,False,...,False,False,False,False,False,False,False,False,True,False


In [15]:
df_encoded=df_encoded.dropna()

In [16]:
df

,Avslag_ind,"ÅDT, total","ÅDT, andel lange kjøretøy",Fartsgrense,"Avkjørsel, holdningsklasse",Funksjonsklasse,Avkjørsler,Trafikkulykker,EGS.BRUKSOMRÅDE.1256,"Kurvatur, horisontal","Kurvatur, stigning"
0,0,100.0,10.0,80.0,Lite streng,E - Lokale adkomstveger,0,0,Bolig,-558.0,-2.7
1,0,850.0,14.0,50.0,Lite streng,E - Lokale adkomstveger,0,4,Bolig,-24.0,8.4
2,0,800.0,10.0,60.0,Mindre streng,D - Lokale samleveger,0,0,Bolig,99999.0,3.7
3,0,6400.0,12.0,50.0,Meget streng,C - Lokale hovedveger,0,23,Industri,99999.0,-0.7
4,0,600.0,14.0,50.0,Lite streng,D - Lokale samleveger,1,2,Bolig,99999.0,-1.8
...,...,...,...,...,...,...,...,...,...,...,...
925,1,1300.0,6.0,30.0,Lite streng,E - Lokale adkomstveger,21,0,Bolig,-1520.0,-0.5
926,0,500.0,10.0,60.0,Mindre streng,D - Lokale samleveger,12,2,Gardsbruk,-1119.0,0.2
927,0,400.0,13.0,60.0,Streng,C - Lokale hovedveger,12,0,Bolig,-192.0,5.5
928,0,1600.0,15.0,80.0,Streng,B - Regionale hovedveger,8,6,Skog/skogbruk,-524.0,-3.0


In [17]:
df_encoded['sving_ind'] = np.where(df_encoded['Kurvatur, horisontal'].abs() > 99000, 0, 1)

C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_29148\354339799.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['sving_ind'] = np.where(df_encoded['Kurvatur, horisontal'].abs() > 99000, 0, 1)


In [18]:
df_encoded['sving'] = np.where(df_encoded['Kurvatur, horisontal'].abs() < 99000, df_encoded['Kurvatur, horisontal'].abs(), 0)

C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_29148\898916879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['sving'] = np.where(df_encoded['Kurvatur, horisontal'].abs() < 99000, df_encoded['Kurvatur, horisontal'].abs(), 0)


In [19]:
df_encoded['bakke_ind'] = np.where(df_encoded['Kurvatur, stigning'].abs() > 0.1, 1, 0)

C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_29148\811313900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['bakke_ind'] = np.where(df_encoded['Kurvatur, stigning'].abs() > 0.1, 1, 0)


In [20]:
df_encoded['sving_sigmoid'] = np.where(df_encoded['Kurvatur, horisontal'].abs() < 99000, 1/(1+np.exp(-0.001*df_encoded['Kurvatur, horisontal'].abs())), 0)

C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_29148\2119347968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['sving_sigmoid'] = np.where(df_encoded['Kurvatur, horisontal'].abs() < 99000, 1/(1+np.exp(-0.001*df_encoded['Kurvatur, horisontal'].abs())), 0)


In [21]:
df_encoded['bakke']=df_encoded['Kurvatur, stigning'].abs()

C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_29148\4044815198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['bakke']=df_encoded['Kurvatur, stigning'].abs()


In [22]:
df_encoded['bakke_sigmoid'] = 1/(1+np.exp(-0.001*df_encoded['Kurvatur, horisontal'].abs()))

C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_29148\567083512.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['bakke_sigmoid'] = 1/(1+np.exp(-0.001*df_encoded['Kurvatur, horisontal'].abs()))


In [23]:
df_encoded['antall_lange_kj']=df_encoded['ÅDT, total']*df_encoded['ÅDT, andel lange kjøretøy']/100

C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_29148\211979289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['antall_lange_kj']=df_encoded['ÅDT, total']*df_encoded['ÅDT, andel lange kjøretøy']/100


In [24]:
df_encoded = df_encoded.drop(['Kurvatur, horisontal', 'Kurvatur, stigning'], axis=1)

In [25]:
y = df_encoded['Avslag_ind']        # target
X = df_encoded.drop(columns=['Avslag_ind'])  # all other columns

In [26]:
X

,"ÅDT, total","ÅDT, andel lange kjøretøy",Fartsgrense,Avkjørsler,Trafikkulykker,"Avkjørsel, holdningsklasse_Lite streng","Avkjørsel, holdningsklasse_Meget streng","Avkjørsel, holdningsklasse_Mindre streng","Avkjørsel, holdningsklasse_Streng",Funksjonsklasse_A - Nasjonale hovedveger,...,EGS.BRUKSOMRÅDE.1256_Service,EGS.BRUKSOMRÅDE.1256_Skog/skogbruk,EGS.BRUKSOMRÅDE.1256_Skole/Barnehage,sving_ind,sving,bakke_ind,sving_sigmoid,bakke,bakke_sigmoid,antall_lange_kj
0,100.0,10.0,80.0,0,0,True,False,False,False,False,...,False,False,False,1,558.0,1,0.635990,2.7,0.635990,10.0
1,850.0,14.0,50.0,0,4,True,False,False,False,False,...,False,False,False,1,24.0,1,0.506000,8.4,0.506000,119.0
2,800.0,10.0,60.0,0,0,False,False,True,False,False,...,False,False,False,0,0.0,1,0.000000,3.7,1.000000,80.0
3,6400.0,12.0,50.0,0,23,False,True,False,False,False,...,False,False,False,0,0.0,1,0.000000,0.7,1.000000,768.0
4,600.0,14.0,50.0,1,2,True,False,False,False,False,...,False,False,False,0,0.0,1,0.000000,1.8,1.000000,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,1300.0,6.0,30.0,21,0,True,False,False,False,False,...,False,False,False,1,1520.0,1,0.820538,0.5,0.820538,78.0
926,500.0,10.0,60.0,12,2,False,False,True,False,False,...,False,False,False,1,1119.0,1,0.753803,0.2,0.753803,50.0
927,400.0,13.0,60.0,12,0,False,False,False,True,False,...,False,False,False,1,192.0,1,0.547853,5.5,0.547853,52.0
928,1600.0,15.0,80.0,8,6,False,False,False,True,False,...,False,True,False,1,524.0,1,0.628083,3.0,0.628083,240.0


In [27]:
poly = PolynomialFeatures(5, include_bias=False, interaction_only=True) 
X = pd.DataFrame(
    poly.fit_transform(X),
    columns=poly.get_feature_names_out(X.columns)
)

In [28]:
X

,"ÅDT, total","ÅDT, andel lange kjøretøy",Fartsgrense,Avkjørsler,Trafikkulykker,"Avkjørsel, holdningsklasse_Lite streng","Avkjørsel, holdningsklasse_Meget streng","Avkjørsel, holdningsklasse_Mindre streng","Avkjørsel, holdningsklasse_Streng",Funksjonsklasse_A - Nasjonale hovedveger,...,sving_ind bakke_ind sving_sigmoid bakke antall_lange_kj,sving_ind bakke_ind sving_sigmoid bakke_sigmoid antall_lange_kj,sving_ind bakke_ind bakke bakke_sigmoid antall_lange_kj,sving_ind sving_sigmoid bakke bakke_sigmoid antall_lange_kj,sving bakke_ind sving_sigmoid bakke bakke_sigmoid,sving bakke_ind sving_sigmoid bakke antall_lange_kj,sving bakke_ind sving_sigmoid bakke_sigmoid antall_lange_kj,sving bakke_ind bakke bakke_sigmoid antall_lange_kj,sving sving_sigmoid bakke bakke_sigmoid antall_lange_kj,bakke_ind sving_sigmoid bakke bakke_sigmoid antall_lange_kj
0,100.0,10.0,80.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,17.171721,4.044828,17.171721,10.921037,609.393844,9581.820108,2257.014238,9581.820108,6093.938443,10.921037
1,850.0,14.0,50.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,...,505.797312,30.468249,505.797312,255.933294,51.616799,12139.135491,731.237984,12139.135491,6142.399063,255.933294
2,800.0,10.0,60.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,6400.0,12.0,50.0,0.0,23.0,0.0,1.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,600.0,14.0,50.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,1300.0,6.0,30.0,21.0,0.0,1.0,0.0,0.0,0.0,0.0,...,32.001001,52.516105,32.001001,26.258053,511.695383,48641.521130,79824.479683,48641.521130,39912.239841,26.258053
852,500.0,10.0,60.0,12.0,2.0,0.0,0.0,1.0,0.0,0.0,...,7.538032,28.410962,7.538032,5.682192,127.167464,8435.057580,31791.866119,8435.057580,6358.373224,5.682192
853,400.0,13.0,60.0,12.0,0.0,0.0,0.0,0.0,1.0,0.0,...,156.685982,15.607436,156.685982,85.840899,316.951012,30083.708634,2996.627746,30083.708634,16481.452600,85.840899
854,1600.0,15.0,80.0,8.0,6.0,0.0,0.0,0.0,1.0,0.0,...,452.219489,94.677068,452.219489,284.031203,620.134793,236963.012152,49610.783440,236963.012152,148832.350319,284.031203


In [29]:
andel_avslag=y.sum()/y.count()

In [30]:
#X=df_with_interactions
#scaler = PowerTransformer()
#X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
# Identify binary dummy columns: only {0,1} or {0.0,1.0}
binary_cols = [
    col for col in X.columns
    if np.isin(X[col].dropna().unique(), [0,1]).all()
]

continuous_cols = [col for col in X.columns if col not in binary_cols]

# Transform only continuous columns
scaler = PowerTransformer()
X_cont_scaled = pd.DataFrame(
    scaler.fit_transform(X[continuous_cols]),
    columns=continuous_cols,
    index=X.index
)

# Combine back into full feature matrix
X = pd.concat([X_cont_scaled, X[binary_cols]], axis=1)

In [ ]:
X

In [ ]:
model = RandomForestClassifier(
    n_estimators=1000,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
model.fit(X, y)

# Get feature importances
importances = pd.Series(model.feature_importances_, index=X.columns)
importances_sorted = importances.sort_values(ascending=False)
# Select top 10 features
top_features = importances_sorted.index[:15]
#X = X[top_features]

In [ ]:
#importances_sorted

In [ ]:
from sklearn.decomposition import PCA

n_components = 100
pca = PCA(n_components=n_components, random_state=42)
X_pca = pca.fit_transform(X)

# Auto-generate names: PC1, PC2, ..., PCn
pc_names = [f"PC{i+1}" for i in range(n_components)]

X_pca_df = pd.DataFrame(X_pca, columns=pc_names, index=X.index)

print("Explained variance ratio:", sum(pca.explained_variance_ratio_))
print("Shape after PCA:", X_pca_df.shape)
print(X_pca_df.head())
##X=X_pca_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
y_train.sum()

In [ ]:
y_test.sum()

In [ ]:
X_train_orig=X_train
y_train_orig=y_train

In [ ]:
smote = ADASYN(sampling_strategy=0.3, random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
# Fit logistic regression
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

In [ ]:
y_test

In [ ]:
yps=model.predict_proba(X_test)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Predicted probabilities for class "1"
y_prob = yps[:, 1]

# Adjustable threshold
threshold = 0.05 # 👈 change this value between 0 and 1

# Convert probabilities to class predictions
y_pred = (y_prob >= threshold).astype(int)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Innvilget", "Avslag"])

# Plot
disp.plot(cmap="Blues")
plt.title(f"Confusion Matrix (threshold = {threshold})")
plt.show()

In [ ]:
# Detailed metrics per class
print("Classification Report Logistic regression:")
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
model = RandomForestClassifier(n_estimators=5000,n_jobs=-1)
model.fit(X_train, y_train)

In [ ]:
yps=model.predict_proba(X_test)

In [ ]:
X_test

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Predicted probabilities for class "1"
y_prob = yps[:, 1]

# Adjustable threshold
threshold = 0.05 # 👈 change this value between 0 and 1

# Convert probabilities to class predictions
y_pred = (y_prob >= threshold).astype(int)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Innvilget", "Avslag"])

# Plot
disp.plot(cmap="Blues")
plt.title(f"Confusion Matrix (threshold = {threshold})")
plt.show()

In [ ]:
# Detailed metrics per class
print("Classification Report Random Forrest:")
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
importances = pd.Series(model.feature_importances_, index=X_train.columns)
importances_sorted = importances.sort_values(ascending=False)

In [ ]:
print(importances_sorted)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Get the top 4 features
top_features = importances_sorted.index[:3]

# Subset data
X_top = X_test[top_features].copy()
X_top['target'] = y_test  # add the target

# Subset data
#X_top = X_train[top_features].copy()
#X_top['target'] = y_train  # add the target


# Pairwise scatter plots
sns.pairplot(
    X_top,
    vars=top_features,
    hue='target',            # color by target
    palette={0: "blue", 1: "red"}, 
    plot_kws={'alpha': 0.7}
)
plt.suptitle("Pairwise Plots of Top Features", y=1.02)
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Sannsynligheter for den positive klassen
y_proba = model.predict_proba(X_test)[:, 1]

# Beregn ROC-verdier
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
auc = roc_auc_score(y_test, y_proba)

# Plott ROC-kurve
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0,1], [0,1], linestyle='--', color='gray')  # baseline
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-kurve - Random Forest')
plt.legend(loc='lower right')
plt.show()

In [ ]:
#Kort sagt: hver x-verdi viser hvor mange negative som blir feil, hver y-verdi viser hvor mange positive som blir riktig.

In [ ]:
y_proba = yps[:, 1]

# Combine into one DataFrame for easy plotting
import pandas as pd
df_plot = pd.DataFrame({'y_test': y_test, 'y_proba': y_proba})

# Plot distributions
plt.figure(figsize=(8,5))
sns.kdeplot(data=df_plot[df_plot['y_test'] == 0]['y_proba'], label='Actual class 0', fill=True)
sns.kdeplot(data=df_plot[df_plot['y_test'] == 1]['y_proba'], label='Actual class 1', fill=True)
plt.title("Distribution of predicted probabilities by actual class")
plt.xlabel("Predicted probability (positive class)")
plt.ylabel("Density")
plt.legend()

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators=1000,sampling_strategy=0.3,
                              random_state=42, n_jobs=-1)
model.fit(X_train_orig, y_train_orig)
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:,1]

In [ ]:
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
df_plot = pd.DataFrame({'y_test': y_test, 'y_proba': y_proba})

# Plot distributions
plt.figure(figsize=(8,5))
sns.kdeplot(data=df_plot[df_plot['y_test'] == 0]['y_proba'], label='Actual class 0', fill=True)
sns.kdeplot(data=df_plot[df_plot['y_test'] == 1]['y_proba'], label='Actual class 1', fill=True)
plt.title("Distribution of predicted probabilities by actual class")
plt.xlabel("Predicted probability (positive class)")
plt.ylabel("Density")
plt.legend()

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.model_selection import PredefinedSplit

# Define the parameter distributions
param_dist = {
    'n_estimators': randint(100, 500),
    'sampling_strategy': uniform(0.3, 0.5)
}

model = BalancedRandomForestClassifier(random_state=42, n_jobs=-1, replacement=True, bootstrap=True)

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=250,                  # number of random combinations to try
    cv=4,
    scoring="average_precision",
    n_jobs=-1,
    verbose=2,
    random_state=42
)

random_search.fit(X_train_orig, y_train_orig)
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)


In [ ]:
y_pred = random_search.predict(X_test)
y_proba = random_search.predict_proba(X_test)[:,1]

In [ ]:
df_plot = pd.DataFrame({'y_test': y_test, 'y_proba': y_proba})

# Plot distributions
plt.figure(figsize=(8,5))
sns.kdeplot(data=df_plot[df_plot['y_test'] == 0]['y_proba'], label='Actual class 0', fill=True)
sns.kdeplot(data=df_plot[df_plot['y_test'] == 1]['y_proba'], label='Actual class 1', fill=True)
plt.title("Distribution of predicted probabilities by actual class")
plt.xlabel("Predicted probability (positive class)")
plt.ylabel("Density")
plt.legend()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Sannsynligheter for den positive klassen
y_proba = random_search.predict_proba(X_test)[:, 1]

# Beregn ROC-verdier
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
auc = roc_auc_score(y_test, y_proba)

# Plott ROC-kurve
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0,1], [0,1], linestyle='--', color='gray')  # baseline
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-kurve - Random Forest')
plt.legend(loc='lower right')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

y_proba = random_search.predict_proba(X_test)[:,1]

# Create a DataFrame for convenience
df = pd.DataFrame({
    'y_true': y_test,
    'y_proba': y_proba
})

# Sort by predicted probability descending
df = df.sort_values('y_proba', ascending=False).reset_index(drop=True)

# Create cumulative counts & proportions
df['cum_true']  = df['y_true'].cumsum()
df['cum_size']  = np.arange(1, len(df)+1)

# Total positives and total size
total_positives = df['y_true'].sum()
total_size      = len(df)

# Compute cumulative gain: proportion of positives captured vs total positives
df['gain'] = df['cum_true'] / total_positives

# Compute population proportion: how many of the cases considered
df['pop_pct'] = df['cum_size'] / total_size

# Plot the gain curve
plt.figure(figsize=(6,6))
plt.plot(df['pop_pct'], df['gain'], label='Model Gain')
plt.plot([0,1], [0,1], linestyle='--', color='grey', label='Random (baseline)')
plt.xlabel('Proportion of sample')
plt.ylabel('Proportion of positives captured')
plt.title('Cumulative Gains Chart')
plt.legend(loc='lower right')
plt.show()


In [ ]:
y_proba = random_search.predict_proba(X_test)[:, 1]
y_true=y_test
df = pd.DataFrame({
    'y_true': y_true,
    'y_proba': y_proba
})

In [ ]:
bins = np.linspace(0, 1, 5)   # 20 intervals

df["bin"] = pd.cut(df["y_proba"], bins)

table = df.groupby("bin").agg(
    n_samples=("y_true", "count"),
    positives=("y_true", "sum"),
    positive_rate=("y_true", "mean"),
)

table = table.sort_index(ascending=False)
table


In [ ]:
y_true.sum()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Predicted probability for positive class
y_proba = random_search.predict_proba(X_test)[:, 1]
y_true  = y_test

df = pd.DataFrame({"y_true": y_true, "y_proba": y_proba})

# Make 20 fixed bins from 0 to 1
bins = np.linspace(0, 1, 5)
df["bin"] = pd.cut(df["y_proba"], bins=bins, include_lowest=True)

# Aggregate: fraction of true positives in each bin
table = df.groupby("bin").agg(
    n_samples=("y_true", "count"),
    positives=("y_true", "sum"),
    positive_rate=("y_true", "mean"),
)

# Compute bin centers for plotting
table["bin_center"] = [interval.mid for interval in table.index]

# Plot
plt.figure(figsize=(7,6))
plt.plot(table["bin_center"], table["positive_rate"], marker="o")
plt.xlabel("Predicted probability (bin center)")
plt.ylabel("Fraction of actual positives")
plt.title("Fraction of positives vs predicted probability (20 bins)")
plt.grid(True)
plt.show()



In [ ]:
y_test.sum()

In [ ]:
y_train.sum()